# MPRT: Find Protein Motifs in Uniprot

In [108]:
import requests

myinput = '/home/hanuman/docs/biomatics/rosalind/MPRT/input-test.txt'
myinput = '/home/hanuman/docs/biomatics/rosalind/MPRT/input-final.txt'
fasta_file = '/home/hanuman/docs/biomatics/rosalind/MPRT/protein.fasta'

In [109]:
def FASTA_iterator(fasta_filename):

    """
    A generator function that reads a FASTA file. At each iteration, the
    function must return a tuple with the format (identifier, sequence).
    """

    with open(fasta_filename, 'rt') as fasta:
        sequence = ''
        identifier = ''
        my_list = []
        for line in fasta:
            if (line[0] == '>'):
                if (sequence != ''):
                    my_tuple = (identifier, sequence)
                    yield(my_tuple)
                    identifier= ''
                    sequence= ''
                identifier = line[1:].strip()
            else:
                sequence += line.strip()
        my_tuple = (identifier, sequence)
        yield(my_tuple)

In [110]:
uniprot_list = []
with open(myinput, 'rt') as f:
    with open(fasta_file, 'wt') as fasta:
        for line in f:
            uniprot_id = line.rstrip()
            uniprot_list.append(uniprot_id)
            r = requests.get('http://www.uniprot.org/uniprot/' + uniprot_id + '.fasta')
            s = str(r.content, 'utf-8')
            fasta.write(s)
fasta_dict = {}
for item in FASTA_iterator(fasta_file):
    uniprot_id = uniprot_list.pop(0)
    fasta_dict[uniprot_id] = item[1]

### N-glycosylation motif is written as N{P}[ST]{P}.

In [111]:
import re
n_glycosylation = re.compile('N[^P]{1}(S|T)[^P]{1}')

In [112]:
output_file = '/home/hanuman/docs/biomatics/rosalind/MPRT/output.txt'

In [113]:
with open(output_file, 'wt') as output:
    for key in fasta_dict:
        s = fasta_dict[key]
        occurrences = []
        for i in range(len(s) - 4):
            w = s[i:i+4]
            for m in n_glycosylation.finditer(w):
                occurrences.append(str(i + 1))
        if occurrences == []:
            continue
        else:
            output.write(key + '\n')
            output.write(' '.join(occurrences))
            output.write('\n') 